### Importación de datos



In [1]:
import pandas as pd
import matplotlib.pyplot as plt

url = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_1%20.csv"
url2 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_2.csv"
url3 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_3.csv"
url4 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_4.csv"

tienda = pd.read_csv(url)
tienda2 = pd.read_csv(url2)
tienda3 = pd.read_csv(url3)
tienda4 = pd.read_csv(url4)

tienda.head()

,Producto,Categoría del Producto,Precio,Costo de envío,Fecha de Compra,Vendedor,Lugar de Compra,Calificación,Método de pago,Cantidad de cuotas,lat,lon
0,Asistente virtual,Electrónicos,164300.0,6900.0,16/01/2021,Pedro Gomez,Bogotá,4,Tarjeta de crédito,8,4.60971,-74.08175
1,Mesa de comedor,Muebles,192300.0,8400.0,18/05/2022,Beatriz Morales,Medellín,1,Tarjeta de crédito,4,6.25184,-75.56359
2,Juego de mesa,Juguetes,209600.0,15900.0,15/03/2021,Juan Fernandez,Cartagena,1,Tarjeta de crédito,1,10.39972,-75.51444
3,Microondas,Electrodomésticos,757500.0,41000.0,03/05/2022,Juan Fernandez,Cali,4,Nequi,1,3.43722,-76.52250
4,Silla de oficina,Muebles,335200.0,20200.0,07/11/2020,Maria Alfonso,Medellín,5,Nequi,1,6.25184,-75.56359


- **Producto y Categoría**: Artículos vendidos y sus calificaciones.
- **Precio y Envío**: Valores de venta y costos asociados.
- **Fecha y ubicación de compra**: Información temporal y geográfica.
- **Evaluación de compra**: Comentarios de clientes.
- **Tipo de Pago y Cuotas**: Métodos utilizados por los clientes.
- **Coordenadas Geográficas**: Ubicación de las transacciones.

In [2]:
import os
import io
import requests
import geopandas as gpd
import folium
import branca.colormap as cm

from PIL import Image
from folium import plugins

tiendas: tuple[pd.DataFrame, ...] = (tienda, tienda2, tienda3, tienda4)
tiendas_nombres: tuple[str, ...] = tuple(f'Tienda {idx + 1}' for idx in range(len(tiendas)))

print('Ventas registradas en cada tienda')
print(*[(nom, df.shape[0]) for nom, df in zip(tiendas_nombres, tiendas)], sep='\n')

Ventas registradas en cada tienda
('Tienda 1', 2359)
('Tienda 2', 2359)
('Tienda 3', 2359)
('Tienda 4', 2358)


In [3]:
def tope(lista: list[pd.Series], *, superior: bool = True) -> int | float:
    """
    Función para hallar el tope superior o inferior de una lista de Series.
    :param lista: Lista de la cual obtener el parámetro, idealmente contiene Series de Pandas.
    :param superior: Si se busca el tope superior (True) o inferior (False).
    :return: Máximo o mínimo de la lista.
    """
    if superior:
        return max(list(map(lambda x: x.max(), lista)))
    else:
        return min(list(map(lambda x: x.min(), lista)))


def get_col(datos: tuple[pd.DataFrame, ...], col: str) -> pd.Series:
    """
    Función para agrupar una columna de distintos DataFrames
    """
    return pd.concat([idx[col] for idx in datos], axis=0)


def save_img(fig: plt.Figure, path: str, fmt: str='svg') -> None:
    """
    Guarda una figura de matplotlib como imagen.
    :param fig: La figura a guardar, debe ser de tipo `plt.Figure`.
    :param path: El nombre del archivo en el que se guardará la imagen.
    :param fmt: Formato para guardar la imagen, por defecto `.svg`
    :return:
    """
    p = './img/'
    if not os.path.exists(p):
        os.makedirs(p)

    fig.savefig(f'{p + path}.{fmt}', format=fmt)
    print(f'Imagen disponible en {p + path}.{fmt}')
    plt.close(fig)


def map_png(m: folium.Map, path: str) -> None:
    """
    Guarda un mapa de Folium como imagen.
    :param m: El mapa a guardar.
    :param path: El nombre del archivo en el que se guardará el mapa.
    :return:
    """
    p = './img/'
    if not os.path.exists(p):
        os.makedirs(p)

    data = m._to_png(size=(1600, 900))
    img = Image.open(io.BytesIO(data))
    img.save(f'{p + path}.png')
    print(f'Imagen disponible en {p + path}.png')

# 1. Análisis de facturación



In [4]:
precios: list[float] = list(map(lambda df: df['Precio'].sum(), tiendas))

In [5]:
def graph1() -> None:
    fig = plt.figure()
    x = tiendas_nombres
    y = list(map(lambda p: p/1e6, precios))

    lim_sup = tope(y)
    lim_inf = tope(y, superior=False)

    color = plt.get_cmap('Pastel1')(range(4))

    plt.bar(x, y, color=color, width=.6, edgecolor='k')

    plt.title('¿Cuántas son las ganancias en las tiendas?', fontsize=14)
    plt.ylabel('Ingresos (en millones)')
    plt.ylim(lim_inf*.95, lim_sup*1.05)
    save_img(fig, '1. Ganancias')


graph1()

Imagen disponible en ./img/1. Ganancias.svg


La tienda 4 es la que menos ganancias genera en comparación a las demás

# 2. Ventas por categoría

In [6]:
def ventas_cat(df) -> pd.Series:
    return (df.groupby('Categoría del Producto')
            .count()['Producto']
            .rename('# Ventas')
            .sort_values(ascending=False))


cat_tiendas: list[pd.Series] = list(map(ventas_cat, tiendas))

In [7]:
def graph2() -> None:
    fig, ax = plt.subplots(2, 2, figsize=(12, 10))
    plt.suptitle('¿Qué categorías se venden más?', fontsize=16)

    for idx in range(len(cat_tiendas)):
        i = 0 if idx < 2 else 1
        j = 0 if idx % 2 == 0 else 1

        ax[i,j].set_title(tiendas_nombres[idx])
        ax[i,j].pie(
            cat_tiendas[idx].values,
            labels=None,
            colors=plt.get_cmap('Set1').colors,
            autopct=lambda pct: str(int(round(pct, 0))) + '%',
            pctdistance=.77,
            wedgeprops=dict(edgecolor='w',
                            linewidth=.9,
                            antialiased=True),
        )

    fig.legend(cat_tiendas[0].index, loc='center')
    plt.tight_layout()
    save_img(fig, '2. Categorías')


graph2()

Imagen disponible en ./img/2. Categorías.svg


Las categorías de productos vendidos muestran resultados similares

# 3. Calificación promedio de la tienda


In [8]:
calificaciones: list[float] = list(map(lambda df: df['Calificación'].mean(), tiendas))

In [9]:
def graph3() -> None:
    fig, ax = plt.subplots(2, 2, figsize=(8, 8))
    fig.suptitle('¿Cómo califican los usuarios a las tiendas?', fontsize=16)

    conteos = list(map(
        lambda df: df.groupby('Calificación').count().sort_values('Calificación', ascending=False)['Producto'],
        tiendas
    ))
    cant_max = tope(conteos)

    # Colores
    nums = list(conteos[0].index)[::-1]
    n = list(map(lambda x: x/len(nums), nums))
    color = plt.get_cmap('autumn_r')(n)

    ticks = list(map(lambda x: str(x) + '★', nums))

    for idx in range(len(calificaciones)):
        i = 0 if idx < 2 else 1
        j = 0 if idx % 2 == 0 else 1

        ax[i,j].bar(conteos[idx].index, conteos[idx].values, width=1, color=color, edgecolor=color[0])
        ax[i,j].set_title(f'Promedio: {calificaciones[idx]:.2f}★')

        ax[i,j].spines[['right', 'top', 'left']].set_visible(False)
        ax[i,j].set_ylim(0, cant_max * 1.05)

        ax[i,j].set_xlabel(tiendas_nombres[idx])
        ax[i,j].set_ylabel('Calificaciones')

        ax[i,j].set_xticks(nums)
        ax[i,j].set_xticklabels(ticks)
        ax[i,j].set_yticks([])

    plt.tight_layout()
    save_img(fig, '3. Calificaciones')


graph3()

Imagen disponible en ./img/3. Calificaciones.svg


Las tiendas 2 y 3 muestran las mejores calificaciones promedio de los usuarios, la diferencia general es menor a 0.1 estrellas, por lo tanto, se asume que la opinión de los clientes es similar entre todas las tiendas

# 4. Productos más y menos vendidos

In [10]:
def top(df, asc=False) -> pd.Series:
    return (df.groupby('Producto')
            .count()['Precio']
            .rename('Vendidos')
            .sort_values(ascending=asc)
            .head(4))


mas_vendidos:   list[pd.Series] = list(map(lambda df: top(df), tiendas))
menos_vendidos: list[pd.Series] = list(map(lambda df: top(df, True), tiendas))

In [11]:
def graph4(lista, superior=True, *, ruta) -> None:
    n_tiendas = len(lista)
    productos = {key: val for key, val in zip(tiendas_nombres, lista)}

    lim_sup = tope(lista)
    lim_inf = tope(lista, superior=False)

    fig, ax = plt.subplots(figsize=(7, 8), layout='constrained')
    titulo = 'Lo que más se vende en cada tienda' if superior else 'Los productos menos adquiridos en las tiendas'
    plt.title(titulo, fontsize=16)

    mult = 0
    height = 1/(n_tiendas+1)
    pos = [y for y in range(n_tiendas)]
    y = pd.Series(pos)

    for idx, item in enumerate(productos.items()):
        labels = [key.index[idx] for key in productos.values()]
        bars = [val.iloc[idx] for val in productos.values()]

        if superior:
            dif = range(len(bars)-(idx+1), n_tiendas*len(bars), len(bars))
        else:
            dif = range(idx, n_tiendas*len(bars), len(bars))
        color = plt.get_cmap('tab20b')(dif)

        offset = height * mult
        rects = ax.barh(y + offset, bars, height=height, label=labels, color=color)
        ax.bar_label(rects, labels, padding=10)
        mult += 1

    ax.set_xlim(lim_inf*.94, lim_sup*1.01)
    ax.set_xlabel('Unidades vendidas')

    ax.set_yticks(y + height*1.5, tiendas_nombres)
    ax.invert_yaxis()

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    save_img(fig, ruta)


graph4(mas_vendidos, ruta='4. Más vendidos')

Imagen disponible en ./img/4. Más vendidos.svg


Los productos más vendidos ayudan a indicar el porqué de la diferencia en las ganancias entre las tiendas: los productos más vendidos en la tienda 1 son muebles y electrodomésticos, productos con un precio de venta por encima de la media general de ventas en las tiendas

In [12]:
graph4(menos_vendidos, False, ruta='4. Menos vendidos')

Imagen disponible en ./img/4. Menos vendidos.svg


Determinar información adicional sobre los productos (e.g., los precios de venta de los productos en cada tienda, o la relación de tipo de producto vs. precio/costo de envío), no es necesario en este análisis

Tampoco es posible calcular las ganancias basadas en crédito debido a la falta de datos sobre tasas de interés o planes de financiación en las tiendas

# 5. Envío promedio por tienda

In [13]:
promedios: list[pd.Series] = list(map(lambda df: df['Costo de envío'].mean(), tiendas))

In [14]:
def graph5() -> None:
    x = [env['Costo de envío'] for env in tiendas]
    x = list(map(lambda f: f/1e3, x))

    colors = ['xkcd:sky', 'xkcd:lime', 'xkcd:vermillion', 'xkcd:lemon yellow']
    labels = [f'{tiendas_nombres[idx]}, Promedio $(\\bar{{x}}): {(lab.item()/1e3):.2f}$ mil' for idx, lab in enumerate(promedios)]

    fig = plt.figure(figsize=(12, 6))
    fig.suptitle('¿Cuánto se gasta por envío en cada tienda?', fontsize=16)

    plt.hist(x, bins=8, histtype='bar', label=labels, color=colors, edgecolor='k', alpha=.9)
    plt.xlabel('Costo de envío (en miles)')
    plt.ylabel('Unidades vendidas')
    plt.legend(fontsize=12)
    save_img(fig, '5. Envíos')


graph5()

Imagen disponible en ./img/5. Envíos.svg


Los gastos de envío en cada tienda no crean una diferencia notable en el precio total de los productos, tampoco representan una diferencia económica para las tiendas, ya que el costo lo asume el cliente

# Extra: Análisis del desempeño geográfico

In [15]:
gis = [t[['Precio', 'Costo de envío', 'Lugar de Compra', 'Calificación', 'lat', 'lon']] for t in tiendas]

tiendas_gis: tuple[gpd.GeoDataFrame, ...] = tuple(
    gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat)) for df in gis
)

# latitud, longitud
lat: pd.Series = get_col(tiendas, 'lat')
lon: pd.Series = get_col(tiendas, 'lon')

In [16]:
def gdf_group(gdf: gpd.GeoDataFrame) -> pd.DataFrame:
    return gdf.groupby(['Lugar de Compra']).agg({
        'Precio': 'mean',
        'Costo de envío': 'mean',
        'Calificación': 'mean',
        'lat': 'count',
        'geometry': 'first',
    }).rename(columns={'lat': 'Ventas'}).sort_values(by='Ventas', ascending=False)


geo_datos = pd.concat([df for df in tiendas_gis])
geo_datos = gdf_group(gpd.GeoDataFrame(geo_datos, geometry=gpd.points_from_xy(geo_datos.lon, geo_datos.lat)))

geo_datos.head()

,Precio,Costo de envío,Calificación,Ventas,geometry
Lugar de Compra,,,,,
Bogotá,460273.471976,24532.082171,4.113619,3943,POINT (-74.08175 4.60971)
Medellín,473927.917027,25282.022472,3.926534,2314,POINT (-75.56359 6.25184)
Cali,503005.064548,26726.812314,4.046673,1007,POINT (-76.5225 3.43722)
Pereira,436996.711799,23211.411992,4.013540,517,POINT (-75.69611 4.81333)
Barranquilla,425937.096774,22692.258065,3.803226,310,POINT (-74.78132 10.96854)


Bogotá, Medellín y Cali son las zonas más importantes para las tiendas

In [17]:
def choropleth_graph() -> folium.Map:
    min_zoom = 6
    max_zoom = 9

    geojson_url = 'https://gist.github.com/john-guerra/43c7656821069d00dcbc/raw/3aadedf47badbdac823b00dbe259f6bc6d9e1899/colombia.geo.json'
    geojson = requests.get(geojson_url).json()

    departamentos = pd.read_json('./data/departamentos.json')['Departamentos']
    departamentos = departamentos.map(lambda c: c.upper())

    fol = folium.Map(
        (lat.median(), lon.median()),
        tiles='CartoDB Voyager',
        min_zoom=min_zoom,
        max_zoom=max_zoom,
        zoom_start=min_zoom,
        zoom_control='topleft',
        font_size='1.5rem'
    )

    geo_departamentos = geo_datos.copy(deep=True).reset_index()
    geo_departamentos['Lugar de Compra'] = geo_departamentos['Lugar de Compra'].map(departamentos)

    folium.Choropleth(
        geo_data=geojson,
        data=geo_departamentos,
        columns=['Lugar de Compra', 'Ventas'],
        key_on='feature.properties.NOMBRE_DPT',
        fill_color='YlOrRd',
        nan_fill_opacity=.1,
        fill_opacity=.5,
        line_opacity=.3,
        legend_name='Ventas en cada departamento'
    ).add_to(fol)
    fol.save('./img/choropleth.html')
    map_png(fol, 'choropleth')
    return fol


choropleth = choropleth_graph()

Imagen disponible en ./img/choropleth.png


In [18]:
def heatmap_graph() -> folium.Map:
    min_zoom = 6
    max_zoom = 9

    fol = folium.Map(
        (lat.median(), lon.median()),
        tiles='Esri WorldGrayCanvas',
        min_zoom=min_zoom,
        max_zoom=max_zoom,
        zoom_start=min_zoom,
        zoom_control='topleft',
        font_size='1.5rem'
    )

    colormap = cm.linear.inferno.to_step(5)
    colormap.caption = 'Compras realizadas en el sector (%)'
    fol.add_child(colormap)

    lin_transform = list(tuple(map(lambda x: x*255, colormap.colors[idx])) for idx in range(len(colormap.colors)))
    lin_map = {key: f'rgba{val}' for key, val in zip(colormap.index, lin_transform)}

    for idx, gdf in enumerate(tiendas_gis):
        heat = [[point.xy[1][0], point.xy[0][0]] for point in gdf.geometry]

        plugins.HeatMap(heat,
                        tiendas_nombres[idx],
                        min_opacity=.3,
                        radius=30,
                        blur=25,
                        gradient=lin_map).add_to(fol)

    fol.save('./img/heatmap.html')
    map_png(fol, 'heatmap')
    return fol


heatmap = heatmap_graph()

Imagen disponible en ./img/heatmap.png


La densidad de ventas por sector muestra una concentración significativa en el sector norte del país